<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree...   8%|███                             |  ETA: 0:00:02

Finished!


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a3,-2.07,3.19,0.73,0.7,2,f,0
3,a4,-2.63,6.97,-0.83,0.6,1,m,a2
4,a5,2.31,3.5,-1.52,0.4,2,m,a2
5,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [5]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [6]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [7]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [8]:
G1 = 1.0
G2 = eye(2)
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6:Run Analysis</button> 

In [9]:
outputMCMCsamples(model1,"x2")
out1=runMCMC(model1,phenotypes,chain_length=5000,output_samples_frequency=100);

A Linear Mixed Model was build using model equations:

y1 = intercept + x1*x3 + x2 + x3 + ID + dam

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1*x3           interaction  fixed                2
x2              factor       random               2
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12

MCMC Information:

methods                        conventional (no markers)
chain_length                                   5000
burnin                                            0
starting_value                                 true
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Hyper-parameters Infor

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [10]:
keys(out1)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of residual variance"
  "Posterior mean of location parameters"

In [11]:
out1["Posterior mean of polygenic effects covariance matrix"]

2×2 Array{Float64,2}:
  3.40035   -0.537862
 -0.537862   2.26884 

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [12]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [13]:
R      = eye(3)
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [14]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [15]:
G1 = eye(2)
G2 = eye(4)
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

INFO: x2 is not found in model equation 1.
INFO: dam is not found in model equation 2.
INFO: dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Run Analysis</button> 

In [16]:
outputMCMCsamples(model2,"x2")
out2=runMCMC(model2,phenotypes,chain_length=5000,output_samples_frequency=100);

A Linear Mixed Model was build using model equations:

y1 = intercept + x1 + x3 + ID + dam
y2 = intercept + x1 + x2 + x3 + ID
y3 = intercept + x1 + x1*x3 + x2 + ID

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1              covariate    fixed                1
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
x2              factor       random               2
x1*x3           interaction  fixed                2

MCMC Information:

methods                        conventional (no markers)
chain_length                                   5000
burnin                                            0
starting_value                                 true
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_ph

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:03


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [17]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "EBV_y2"
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "EBV_y3"
  "Posterior mean of residual variance"
  "Posterior mean of location parameters"

In [18]:
out2["Posterior mean of polygenic effects covariance matrix"]

4×4 Array{Float64,2}:
  2.22204   -0.307771   -0.376363   -0.177532 
 -0.307771   1.31058     0.0275585   0.171411 
 -0.376363   0.0275585   1.0198     -0.0389204
 -0.177532   0.171411   -0.0389204   1.38742  